## Decision Tree Model

### Imports and Utils

In [1]:
"""
Importing the necessary libraries
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.special import xlogy
from sklearn import datasets
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Import graphviz
try:
    from graphviz import Digraph
except ModuleNotFoundError:
    %pip install graphviz
    from graphviz import Digraph

# Remove all the warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
"""
Utility functions for the Model
"""

def entropy(Y: pd.Series) -> float:
    """
    Y: pd.Series: Output values

    Returns: float: Entropy
    """

    vals = Y.value_counts(normalize=True)
    return -np.sum(xlogy(vals, vals))

def gini_index(Y: pd.Series) -> float:
    """
    Y: pd.Series: Output values

    Returns: float: Gini Index
    """

    vals = Y.value_counts(normalize=True)
    return 1 - np.sum(np.square(vals))

def information_gain(parent: pd.Series, left: pd.Series, right: pd.Series):
    """
    parent: pd.Series: Input parent dataset.
    left: pd.Series: Subset of the parent dataset.
    right: pd.Series: Subset of the parent dataset.

    Returns: float: Information gain.
    """

    # calculate parent and child entropy
    before_entropy = entropy(parent)
    after_entropy = (len(left) / len(parent)) * entropy(left) + (len(right) / len(parent)) * entropy(right)
        
    # calculate information gain 
    information_gain = before_entropy - after_entropy
    return information_gain

def best_split(dataset: pd.DataFrame, num_samples: int, num_features: int):
    """
    dataset: pd.DataFrame: The dataset to split.
    num_samples: int: The number of samples in the dataset.
    num_features: int: The number of features in the dataset.

    Returns: dict: A dictionary with the best split.
    """
        
    # Find the best split
    best_split = {'gain': -1, 'feature': None, 'threshold': None, "left_dataset": None, "right_dataset": None}
    for feature_index in range(num_features):
        feature_values = dataset.iloc[:, feature_index]
        thresholds = np.unique(feature_values)
        for threshold in thresholds:
            left_dataset, right_dataset = split_data(dataset, feature_index, threshold)
            y, left_y, right_y = dataset.iloc[:, -1], left_dataset.iloc[:, -1], right_dataset.iloc[:, -1]
            gain = information_gain(y, left_y, right_y)
            if gain > best_split["gain"]:
                best_split["gain"] = gain
                best_split["feature"] = feature_index
                best_split["threshold"] = threshold
                best_split["left_dataset"] = left_dataset
                best_split["right_dataset"] = right_dataset
    return best_split

def split_data(dataset: pd.DataFrame, feature: int, threshold: float):
    """
    dataset: pd.DataFrame: Input dataset.
    feature: int: Index of the feature to be split on.
    threshold: float: Threshold value to split the feature on.

    Returns:
        left_dataset: pd.DataFrame: Subset of the dataset.
        right_dataset: pd.DataFrame: Subset of the dataset.
    """
    # Create mask of the dataset using threshold
    mask = (dataset.iloc[:, feature] <= threshold)

    # Mask the dataset
    left_dataset = dataset[mask]
    right_dataset = dataset[~mask]
    return left_dataset, right_dataset

### Dataset Loading and Preprocessing

In [3]:
# Load the dataset
iris = datasets.load_iris()

# Convert to DataFrame
dataset = pd.DataFrame(data=iris.data, columns=iris.feature_names)
dataset['target'] = iris.target

# Print the first few records
print(dataset.head())

# Print the size of the dataset
print("Size of the dataset: ", dataset.shape)

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target  
0       0  
1       0  
2       0  
3       0  
4       0  
Size of the dataset:  (150, 5)


In [4]:
# Extract X and Y from dataset
X = dataset.iloc[:, :-1]
Y = dataset.iloc[:, -1]

# Normalize the dataset
X = pd.DataFrame(preprocessing.normalize(X), columns=X.columns)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8, random_state=42)

# Print the first few records
print(f'Feature Train Dataset:\n{X_train.head()}\nsize: {X_train.shape}\n')
print(f'Target Train Dataset:\n{Y_train.head()}\nsize: {Y_train.shape}\n')
print(f'Feature Test Dataset:\n{X_test.head()}\nsize: {X_test.shape}\n')
print(f'Target Test Dataset:\n{Y_test.head()}\nsize: {Y_test.shape}\n')

Feature Train Dataset:
    sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
22           0.775771          0.607125           0.168646          0.033729
15           0.773811          0.597328           0.203635          0.054303
65           0.769454          0.356016           0.505313          0.160782
11           0.786991          0.557452           0.262330          0.032791
42           0.786090          0.571702           0.232254          0.035731
size: (120, 4)

Target Train Dataset:
22    0
15    0
65    1
11    0
42    0
Name: target, dtype: int32
size: (120,)

Feature Test Dataset:
     sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
73            0.736599          0.338111           0.567543          0.144905
18            0.806828          0.537885           0.240633          0.042465
118           0.706006          0.238392           0.632655          0.210885
78            0.733509          0.354530           0.550132   

### Model Creation and Training

In [5]:
class Node():
    """
    A class representing a node in a decision tree.
    """

    def __init__(self, feature=None, threshold=None, left=None, right=None, gain=None, value=None):
        """
        feature: string: The feature used for splitting at this node.
        threshold: List of float: The threshold used for splitting at this node.
        left: Node: Pointer to the left Node.
        Right: Node: Pointer to the Right Node.
        gain: float: The gain of the split.
        value: float: predicted value at this node.
        """

        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.gain = gain
        self.value = value

In [6]:
class DecisionTree():
    """
    A decision tree classifier.
    """

    def __init__(self, min_samples=2, max_depth=2):
        """
        Constructor for DecisionTree class.

        min_samples: int: Minimum number of samples at leaf node.
        max_depth: int: Maximum depth of the decision tree.
        """
        self.min_samples = min_samples
        self.max_depth = max_depth
    
    def build_tree(self, dataset: pd.DataFrame, current_depth=0):
        """
        dataset: pd.DataFrame: The dataset to build the tree.
        current_depth: int: The current depth of the tree.

        Returns: Node: The root node of the decision tree.
        """
        
        # split the dataset into X, y values
        X, y = dataset.iloc[:, :-1], dataset.iloc[:, -1]
        n_samples, n_features = X.shape
        
        # Terminating conditions
        if n_samples >= self.min_samples and current_depth <= self.max_depth:
            best_split_values = best_split(dataset, n_samples, n_features)
            left_node = self.build_tree(best_split_values["left_dataset"], current_depth + 1)
            right_node = self.build_tree(best_split_values["right_dataset"], current_depth + 1)

            return Node(best_split_values["feature"], best_split_values["threshold"], left_node, right_node, best_split_values["gain"])

        # compute leaf node value
        leaf_value = y.mean()
        return Node(value=leaf_value)
    
    def fit(self, X: pd.DataFrame, y: pd.Series):
        """
        X: pd.DataFrame: The feature datset.
        y: pd.Series: The target values.
        """
        
        dataset = pd.concat([X, y], axis=1) 
        self.root = self.build_tree(dataset)

    def predict(self, X: pd.DataFrame):
        """
        X: pd.DataFrame: The feature matrix to make predictions for.

        Returns:
        list: A list of predicted class labels.
        """
        
        predictions = X.apply(self.traverse_tree, axis=1, args=(self.root,))
        return predictions
    
    def traverse_tree(self, X: pd.Series, node: Node):
        """
        X: pd.Series: The feature vector to predict the target value for.
        node: Node: The current node being evaluated.

        Returns: float: The predicted target value.
        """
        
        if node.value != None: # if the node is a leaf node
            return node.value
        else: # if the node is not a leaf node
            feature = X.iloc[node.feature]
            if feature <= node.threshold:
                return self.traverse_tree(X, node.left)
            else:
                return self.traverse_tree(X, node.right)
            
    def plot_tree(self, node=None, depth=0, dot=None):
        """
        Plot the decision tree.
        """
        if dot is None:
            dot = Digraph()
            dot.attr('node', shape='box')
            dot.node(name=str(node), label=str(node.value))
        else:
            if node is not None:
                dot.node(name=str(node) + str(depth), label=str(node.value))
                if node.left is not None:
                    dot.edge(str(node) + str(depth), str(node.left) + str(depth+1))
                    self.plot_tree(node.left, depth+1, dot)
                if node.right is not None:
                    dot.edge(str(node) + str(depth), str(node.right) + str(depth+1))
                    self.plot_tree(node.right, depth+1, dot)
        return dot

In [7]:
# Defining the model for Decision Tree
model = DecisionTree(min_samples=2, max_depth=2)

In [8]:
# Training the model
model.fit(X_train, Y_train)

# Calculating the metrics
Y_pred = model.predict(X_train)
print(f"Train MSE: {mean_squared_error(Y_train, Y_pred)}")

Train MSE: 0.02321428571428571


### Testing and Plotting

In [9]:
# Predicting the values
Y_pred = model.predict(X_test)

# Calculating the metrics
print(f'Test MSE: {mean_squared_error(Y_test, Y_pred)}')

Test MSE: 0.15884353741496596
